In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import torch

from matplotlib import cm
from torch.utils.data import DataLoader, Dataset


from counterfactuals.datasets import LawDataset, MoonsDataset
from counterfactuals.generative_models import MaskedAutoregressiveFlowDistance as MaskedAutoregressiveFlow
from counterfactuals.discriminative_models import (
    LogisticRegression,
    MultilayerPerceptron,
)
from counterfactuals.metrics import CFMetrics

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

In [ ]:
class PairDistanceDataset(Dataset):
    def __init__(self, class_zero, class_one, length=None):
        """
        Initialize with two arrays, one for each class.
        """
        self.length = length
        self.class_zero = torch.tensor(class_zero, dtype=torch.float32).to(device)  # ✅ Move to CUDA
        self.class_one = torch.tensor(class_one, dtype=torch.float32).to(device)  # ✅ Move to CUDA

        # Calculate pairwise distances between zero and one classes
        self.zero_one_distance = torch.cdist(self.class_zero, self.class_one) ** 4

        self.size_zero = class_zero.shape[0]
        self.size_one = class_one.shape[0]

    def __len__(self):
        # The total combinations are len(class_zero) * len(class_one)
        if self.length is not None:
            return self.length
        return self.size_zero * self.size_one

    def get_specific_item(self, idx):
        """
        Get the specific item based on the index.
        Sample the second point based on the distance weight from another class.
        """
        if idx < self.size_zero:
            i = idx
            x_orig = self.class_zero[i]

            # Calculate weights for sampling y
            zero_one_weight = 1 / self.zero_one_distance[i]
            zero_one_weight /= zero_one_weight.sum()

            j = torch.multinomial(zero_one_weight, num_samples=1).item()
            x_cf = self.class_one[j]
        else:
            i = idx + self.size_zero
            x_orig = self.class_one[i]

            # Calculate weights for sampling y
            zero_one_weight = 1 / self.zero_one_distance[:, i]
            zero_one_weight /= zero_one_weight.sum()

            j = torch.multinomial(zero_one_weight, num_samples=1).item()
            x_cf = self.class_zero[j]
        return torch.tensor(x_cf, dtype=torch.float32), torch.tensor(
            x_orig, dtype=torch.float32
        )

    def __getitem__(self, idx):
        """
        Randomly select a point from one class.
        Sample the second point based on the distance weight from another class.
        """
        if torch.rand(1) > 0.5:
            i = torch.randint(0, self.size_zero, (1,)).item()
            x_orig = self.class_zero[i]

            # Calculate weights for sampling y
            zero_one_weight = 1 / self.zero_one_distance[i]
            zero_one_weight /= zero_one_weight.sum()

            j = torch.multinomial(zero_one_weight, num_samples=1).item()
            x_cf = self.class_one[j]
        else:
            i = torch.randint(0, self.size_one, (1,)).item()
            x_orig = self.class_one[i]

            # Calculate weights for sampling y
            zero_one_weight = 1 / self.zero_one_distance[:, i]
            zero_one_weight /= zero_one_weight.sum()

            j = torch.multinomial(zero_one_weight, num_samples=1).item()
            x_cf = self.class_zero[j]
        return torch.tensor(x_cf, dtype=torch.float32), torch.tensor(
            x_orig, dtype=torch.float32
        )

In [ ]:
from counterfactuals.datasets import GermanCreditDataset, AdultDataset

# dataset = MoonsDataset(file_path="../../data/moons.csv")
# dataset = LawDataset(file_path="../../data/law.csv")
# dataset = GermanCreditDataset(file_path="data/german_credit.csv")
dataset = AdultDataset(file_path="data/adult.csv")

In [ ]:
from counterfactuals.generative_models import MaskedAutoregressiveFlow as baseMAF
dataset.X_train = np.array(dataset.X_train, dtype=np.float32)
dataset.y_train = np.array(dataset.y_train, dtype=np.float32)
dataset.X_test = np.array(dataset.X_test, dtype=np.float32)
dataset.y_test = np.array(dataset.y_test, dtype=np.float32)



flow_train_dataloader = dataset.train_dataloader(
    batch_size=128, shuffle=True, noise_lvl=0.03
)
flow_test_dataloader = dataset.test_dataloader(batch_size=128, shuffle=False)
flow = baseMAF(
    features=dataset.X_test.shape[1],
    hidden_features=16,
    num_blocks_per_layer=4,
    num_layers=8,
    context_features=1,
    device=device
).to(device)
flow.fit(flow_train_dataloader, flow_test_dataloader, num_epochs=1000, patience=50)

In [ ]:
disc_model = MultilayerPerceptron(
    input_size=29,  # Adjust based on the actual feature size of your dataset
    hidden_layer_sizes=[256, 256],
    target_size=1,
    dropout=0.2,
    device=device
).to(device)

train_dataloader = dataset.train_dataloader(batch_size=64, shuffle=True, noise_lvl=0.0)
test_dataloader = dataset.test_dataloader(batch_size=64, shuffle=False)
disc_model.fit(train_dataloader, test_dataloader, epochs=10000, patience=100, lr=1e-3)

# validate
y_pred = disc_model.predict(dataset.X_test).cpu().detach().numpy()  # ✅ Fix added

y_true = dataset.y_test
print(f"Accuracy: {np.mean(y_pred == y_true)}")
# disc_model.load("../models/MoonsDataset/disc_model_MultilayerPerceptron.pt")

disc_model.eval()
dataset.y_train = disc_model.predict(dataset.X_train).cpu().detach().numpy()
dataset.y_test = disc_model.predict(dataset.X_test).cpu().detach().numpy()

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

# Plot training dataset
axes[0].scatter(dataset.X_train[:, 0], dataset.X_train[:, 1], c=dataset.y_train, cmap='viridis', marker='o')
axes[0].set_title('Training Dataset')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# Plot test dataset
axes[1].scatter(dataset.X_test[:, 0], dataset.X_test[:, 1], c=dataset.y_test, cmap='viridis', marker='o')
axes[1].set_title('Test Dataset')
axes[1].set_xlabel('Feature 1')
axes[1].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

In [ ]:
class_zero = dataset.X_train[dataset.y_train == 0]
class_one = dataset.X_train[dataset.y_train == 1]

pair_dataset_train = PairDistanceDataset(class_zero, class_one, length=5000)


def collate_fn(batch):
    X, y = zip(*batch)
    X = torch.stack(X).to(device)
    y = torch.stack(y).to(device)
    noise = torch.randn_like(X) * 0.03
    noise = torch.randn_like(y) * 0.03
    X = X + noise
    y = y + noise
    return X, y


train_dataloader = DataLoader(
    pair_dataset_train, batch_size=256, shuffle=True, collate_fn=collate_fn
)
train_dataloader = DataLoader(pair_dataset_train, batch_size=128, shuffle=True)


In [ ]:
class_zero = dataset.X_test[dataset.y_test == 0]
class_one = dataset.X_test[dataset.y_test == 1]

pair_dataset_test = PairDistanceDataset(class_zero, class_one)

test_dataloader = DataLoader(pair_dataset_test, batch_size=2048, shuffle=False)

In [ ]:
cf = MaskedAutoregressiveFlow(
    features=dataset.X_test.shape[1],
    hidden_features=16,
    num_blocks_per_layer=2,
    num_layers=2,
    context_features=dataset.X_test.shape[1],
    device=device
).to(device)
cf.fit(
    train_dataloader, test_dataloader, num_epochs=1000, learning_rate=1e-3, patience=100, lambda_dist=0.2, checkpoint_path="best_cf_model_dist_0.2_adult_cuda.pt"
)
cf.load("best_cf_model_dist_0.2_adult_cuda.pt")

In [ ]:
from tqdm import tqdm

cfs = []
with torch.no_grad():
    for x in dataset.X_test:
        points, log_prob = cf.sample_and_log_prob(
            100, context=torch.from_numpy(np.array([x]))
        )
        cfs.append(points)
cfs = torch.stack(cfs).squeeze().permute(1, 0, 2).cpu().numpy()

all_metrics = []
for i in tqdm(range(cfs.shape[0])):
    metrics = CFMetrics(
        X_cf=cfs[i],
        y_target=np.abs(dataset.y_test - 1),
        X_train=dataset.X_train,
        y_train=dataset.y_train,
        X_test=dataset.X_test,
        y_test=dataset.y_test,
        gen_model=flow,
        disc_model=disc_model,
        continuous_features=dataset.numerical_features,
        categorical_features=dataset.categorical_features,
        prob_plausibility_threshold=1.2,
    )

    all_metrics.append(metrics.calc_all_metrics())

# Calculate mean and standard deviation for each metric
mean_metrics = {key: np.mean([m[key] for m in all_metrics]) for key in all_metrics[0]}
std_metrics = {key: np.std([m[key] for m in all_metrics]) for key in all_metrics[0]}

# Print the results
for key in mean_metrics:
    print(f"{key}: {mean_metrics[key]:.4f} ± {std_metrics[key]:.4f}")

In [ ]:
import pandas as pd
from metrics import distance, feasibility, constraint_violation, success_rate
cfs = cfs.reshape(-1, cfs.shape[-1])  # Flatten first two dimensions

distance_pd = pd.DataFrame(distance(cfs, dataset.y_test, dataset))

feasibility_pd = pd.DataFrame(feasibility(cfs, dataset, dataset.df.columns), columns=['feasibility'])

# const_pd = pd.DataFrame(constraint_violation(decoded_cfs, decoded_factuals, dataset), columns=['violation'])

success_pd = pd.DataFrame(success_rate(cfs[dataset.df.columns], cf), columns=['success'])

In [ ]:
results = pd.concat([distance_pd, feasibility_pd, success_pd], axis=1)
print(results)

In [ ]:
results